In [31]:
import tensorflow as tf
from tensorflow import keras

## 误差计算

## 均方差误差函数
均方差（Mean Squared Error，简称MSE），把输出向量和真实向量映射到笛卡尔坐标系的两个点上，通过计算这两个点之间的欧式距离（准确地说是欧式距离的平方）来衡量两个向量之间的差距：


$$\displaystyle MSE(y,o)\stackrel{\mathrm{\Delta}}{=}\frac{1}{d_{out}}\sum^{d^{out}}_{i=1}(y_i-o_i)$$

当MSE函数达到最小值0时，输出等于真实标签，此时神经网络的参数达到最优状态

In [32]:
# 构造网络输出
o = tf.random.normal([2, 10]) 
print(o)

# 构造真实值
y_onehot = tf.constant([1,3])  
y_onehot = tf.one_hot(y_onehot, depth=10)
print(y_onehot)

# 计算每个样本的均方差
loss = keras.losses.MSE(y_onehot, o) 
loss

tf.Tensor(
[[ 2.1270916   1.692834   -0.9886939   0.46138158  0.35399404 -0.7563028
  -1.2045842  -2.8295927   1.5409844  -1.0450587 ]
 [ 0.91720855 -0.16368659 -0.25263077 -1.1987484   0.07164879 -0.6429402
   0.8043688  -1.1519246  -0.62885076  0.04400402]], shape=(2, 10), dtype=float32)
tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]], shape=(2, 10), dtype=float32)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.9816631 , 0.85562164], dtype=float32)>

In [33]:
# 计算平均样本的均方差
loss = tf.reduce_mean(loss)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=1.4186424>

In [34]:
# 通过层方式计算均方差（调用__call_函数即可完成前向计算）
criteon = keras.losses.MeanSquaredError()
loss = criteon(y_onehot, o)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=1.4186424>

### 交叉熵误差函数

#### 熵(Entropy)
熵越大，代表不确定性越大，信息量也就越大。某个 分布𝑃(𝑖)的熵定义为

$$H(P)\stackrel{\mathrm{\Delta}}{=}-\sum_{i=1}𝑃(i)log_2𝑃(i)$$
举个例子，对于 4 分类问题，如果某个 样本的真实标签是第 4 类，那么标签的 One-hot 编码为[0,0,0,1]，即这张图片的分类是唯一 确定的，它属于第 4 类的概率𝑃(𝑦为 4|𝒙) = 1，不确定性为 0，它的熵可以简单的计算为:
$$−0 ∙ log2 0 − 0 ∙ log2 0 − 0 ∙ log2 0 − 1 ∙ log2 1 = 0$$
也就是说，对于确定的分布，熵为 0，不确定性最低。 如果它预测的概率分布是[0.1,0.1,0.1,0.7]，它的熵可以计算为:
$$−0.1 ∙ log2 0.1 − 0.1 ∙ log2 0.1 − 0.1 ∙ log2 0.1 − 0.7 ∙ log2 0.7 ≈ 1.356$$
这种情况比前面确定性类别的例子的确定性要稍微大点。

考虑随机分类器，它每个类别的预测概率是均等的:[0.25,0.25,0.25,0.25]，同样的方
法，可以计算它的熵约为 2，这种情况的不确定性略大于上面一种情况。

In [14]:
# 计算随机分类器的熵（每个类别预测概率均等）
a = tf.fill([4], 0.25) 
print(a)
print(a*tf.math.log(a)/tf.math.log(2.))
-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.))


tf.Tensor([0.25 0.25 0.25 0.25], shape=(4,), dtype=float32)
tf.Tensor([-0.5 -0.5 -0.5 -0.5], shape=(4,), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [24]:
# 计算概率分布为0.1,0.1,0.1,0.7的熵
a = tf.constant([0.1, 0.1, 0.1, 0.7])
print(-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.)))

# 计算概率分布为0.01, 0.01, 0.01, 0.97的熵
a = tf.constant([0.01, 0.01, 0.01, 0.97])
print(-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.)))


tf.Tensor(1.3567797, shape=(), dtype=float32)
tf.Tensor(0.24194068, shape=(), dtype=float32)


#### 交叉熵(Cross Entropy)
$$H(p||q)\stackrel{\mathrm{\Delta}}{=}-\sum{p(i)log_2q(i)}$$
通过变换，交叉熵可以分解为p的熵H(p)和p与q的KL散度(Kullback-Leibler Divergence)的和：
$$H(p||q)=H(p)+D_{KL}(p||q)$$
其中KL定义为：
$$D_{KL}(p||q)=\sum{p(i)log\Big(\frac{p(i)}{q(i)}\Big)}$$
KL 散度是 Solomon Kullback 和 Richard A. Leibler 在 1951 年提出的用于衡量 2 个分布之间 距离的指标。𝑝 = 𝑞时，$𝐷_{𝐾𝐿}(𝑝||𝑞)$取得最小值0，𝑝与𝑞之间的差距越大，$𝐷_{𝐾𝐿}(𝑝||𝑞)$也越 大。

交叉熵可以很好地衡量2个分布之间的“距离”。特别地，当分类总是中y的编码分布采用One-hot编码y时：H(p) = 0，此时
$$H(p||q) = H(p) + D_{KL}(p||q) = D_{KL}(p||q)$$
退化到真实标签分布y与输出概率分布o之间的KL散度上。

根据KL散度的定义，我们推导分类总是中交叉熵的计算表达式：
$$
\begin{align*}
H(p||q) = D_{p||q} &= \sum_j{y_ilog\left(\frac{y_j}{o_j}\right)} \\
&=1·log\frac{1}{o_i} + \sum_{j \neq i}0·log\left(\frac{0}{o_j}\right) \\
&= -logo_i
\end{align*}
$$
其中i为One-hot编码中为1的索引号，也是当前输入的真实类别。

可以看到，L只与真实 类别𝑖上的概率$𝑜_𝑖$有关，对应概率$𝑜_𝑖$越大，𝐻(𝑝||𝑞)越小。当对应类别上的概率为 1 时，交叉 熵𝐻(𝑝||𝑞)取得最小值 0，此时网络输出𝒐与真实标签𝒚完全一致，神经网络取得最优状态。

因此最小化交叉熵损失函数的过程也是最大化正确类别的预测概率的过程。从这个角 度去理解交叉熵损失函数，非常地直观易懂。



举例说明加深理解：Classification问题

$P_1 = [1\quad 0\quad 0\quad 0\quad 0]$, 真实值，5张图片，第1张是狗，其他的都不是。

**预测一：**

$Q_1 = [0.4\quad 0.3\quad 0.05\quad 0.05\quad 0.2]$，预测概率

$$
\begin{align*}
H(P_1, Q_1) &= -\sum_iP_1(i)logQ_1(i) \\
&= -(1log0.4 + 0log0.3 + 0log0.05 + 0log0.05 + 0log0.2) \\
&= -log0.4 \\
&\approx 0.916
\end{align*}
$$

交叉熵越大，预测越不准确

**预测二：** 

$Q_1 = [0.98\quad 0.01\quad 0\quad 0\quad 0.01]$，预测概率

$$
\begin{align*}
H(P_1, Q_1) &= -\sum_iP_1(i)logQ_1(i) \\
&= -(1log0.98 + 0log0.01 + 0log0 + 0log0 + 0log0.01) \\
&= -log0.98 \\
&\approx 0.02
\end{align*}
$$

交叉熵越小，预测越准确








##### Categorical Cross Entropy

In [27]:

tf.losses.categorical_crossentropy([0,1,0,0], [0.25, 0.25, 0.25, 0.25])

<tf.Tensor: shape=(), dtype=float32, numpy=1.3862944>

In [28]:
tf.losses.categorical_crossentropy([0,1,0,0], [0.1, 0.1, 0.8, 0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3978953>

In [29]:
tf.losses.categorical_crossentropy([0,1,0,0], [0.01, 0.97, 0.01, 0.01])

<tf.Tensor: shape=(), dtype=float32, numpy=0.030459179>

In [35]:
criteon([0,1,0,0], [0.01, 0.97, 0.01, 0.01])

<tf.Tensor: shape=(), dtype=float32, numpy=0.00029999955>

In [36]:
tf.losses.BinaryCrossentropy()([1], [0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025842>

In [39]:
tf.losses.binary_crossentropy([1], [0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025842>